# Setting Up a PEERING Client on FABRIC

This document will guide you through setting up a [PEERING](https://peering.ee.columbia.edu) client on FABRIC.  PEERING runs on its own slice on FABRIC, and your client will run on a slice you control.  Connectivity between your client's slice and PEERING's slice will happen over OpenVPN tunnels traversing FABRIC's internal network.  The OpenVPN tunnels will operate like a direct link between your slice and the PEERING routers on FABRIC.  We will then configure the
[BIRD](https://bird.network.cz) to establish BGP sessions and exchange routes with PEERING.

For more in-depth information on PEERING, check out our [CoNEXT Paper](https://homepages.dcc.ufmg.br/~cunha/papers/schlinker19peering.pdf) and the [PEERING client's README](https://github.com/PEERINGTestbed/client/blob/master/README.md) file.

## PEERING Experiment Approval

Experiments on PEERING go through a review process that checks they follow PEERING's accepted-use-policy and will not disrupt the Internet.  Head over to [PEERING's website](https://peering.ee.columbia.edu), create an account, and submit an experiment proposal.  Once your experiment is approved, it will be allocated IP prefixes from PEERING address space and you will receive certificates that will allow you to establish OpenVPN tunnels (and BGP sessions over the tunnels) with PEERING routers.

## Connectivity Overview

The PEERING slice and your slice will communicate over FABRIC's internal network.  A slice can connect to FABRIC's internal network by requesting FABnet interfaces.  A FABnetv6 interface is allocated a /64 prefix inside `2602:fcfb::/40`.  To allow your slice's FABnet to speak with PEERING's FABnet, both slices need to route the /40 superprefix over the FABnet interface.

![Deployment Overview](./figs/peering-fabric.png)

In [ ]:
import ipaddress
import json
import os
import pathlib
import time

from fabrictestbed_extensions.fablib.fablib import FablibManager

manager = FablibManager()
print(f"FABRIC's FABNet4 superprefix: {FablibManager.FABNETV4_SUBNET}")
print(f"FABRIC's FABNet6 superprefix: {FablibManager.FABNETV6_SUBNET}")

If the command above fails, you may need to update your version of `fabrictestbed-extensions` (this notebook has last been tested with 1.5.4).  Set the version number inside `requirements.txt` and run

```bash
pip install --upgrade -r requirements.txt
```

In [ ]:
manager.show_config();

## Setting Up the Slice

We create a FABnetv6 named `fab6` that we will use to connect to PEERING's routers.  OpenVPN will be the main application using this interface, and the BGP sessions as well as traffic will go over the OpenVPN tunnels.

> The Ubuntu 22.04 image has been tested to work.  Using a different distros or versions should work, but may incur in compabitility issues due to different software versions.  In particular, the provided configuration files work for BIRD 1 and Openvpn 2.

> Below we will create one node at the Washington FABRIC site, but you can deploy any number of clients at any site within FABRIC.

The cells below will configure a client running on the `WASH` site. You can run as many clients as you want on any site you want.

In [3]:
fabsite = "WASH"
slice_name = f"slice_{fabsite.lower()}"
node_name = f"client_{fabsite.lower()}"
fabnet_name = f"client_{fabsite.lower()}_fab6"

In [ ]:
slice = manager.new_slice(name=slice_name)
node = slice.add_node(name=node_name,
                     site=fabsite,
                     instance_type='fabric.c2.m8.d100',
                     image='default_ubuntu_22')
wash_fab6 = node.add_component(model="NIC_Basic", name=fabnet_name).get_interfaces()[0]
slice.add_l3network(name=fabnet_name, interfaces=[wash_fab6], type="IPv6")
slice.submit()

## Configuring the FABnet Interface

By default, the FABnetv6 interface we just created is not configured.  We will set it up with the first IP address in the its allocated subnet, and then create a route for the /40 superprefix.

In [ ]:
slice = manager.get_slice(name=slice_name)
node = slice.get_node(name=node_name)
fab6iface = node.get_interface(network_name=fabnet_name)
fab6net = slice.get_network(name=fabnet_name)
fab6pfx = fab6net.get_subnet()
fab6ips = fab6net.get_available_ips()
# Verifying assumptions (.1 is gateway and omitted from get_available_ips):
generator = fab6pfx.hosts()
assert fab6net.get_gateway() == next(generator)
assert fab6ips[0] == next(generator)
print(f"Our FABnetv6's prefix is {fab6pfx}")
print(f"Our FABnetv6's IP will be {fab6ips[0]}")
print(f"Our FABnetv6's gateway is {fab6net.get_gateway()}")

In [ ]:
node.get_ssh_command()

In [ ]:
fab6json = json.loads(fab6iface.show(output="json"))
node.execute(f"sudo ip link set dev {fab6json['dev']} up")
fab6iface.ip_addr_add(fab6ips[0], subnet=fab6pfx)
node.ip_route_add(subnet=FablibManager.FABNETV6_SUBNET, gateway=fab6net.get_gateway());

If you want to check configuration is correct, the commands below should:

1. Show that the interface is `up`
2. That the IPv6 address is configured
3. That a route to the /40 superprefix exists
4. And that `fabwash.atlas.peering.ee.columbia.edu` is reachable

In [ ]:
node.execute(f"ip link show dev {fab6json['dev']}")
node.execute(f"ip addr show dev {fab6json['dev']}")
node.execute(f"ip -6 route")
node.execute(f"ping -c1 fabwash.atlas.peering.ee.columbia.edu")

## One-time Configuration of the PEERING Client

Now that your node can communicate with the PEERING slice, you will need to configure the PEERING client on your node.  This is a one-time process once the VM is created (although you will need to reexecute it if the VM is recreated).  First, configure [SSH access][fabric-ssh] and [NAT64][fabric-nat64] on your node following FABRIC's instructions.  With SSH access to your node and NAT64 working, follow these steps:

[fabric-ssh]: ../../configure.ipynb
[fabric-nat64]: ../fablib_api/accessing_ipv4_services_from_ipv6_nodes/accessing_ipv4_services_from_ipv6_nodes.ipynb
1. Install required dependencies

    ```bash
    apt update
    apt install git openvpn bird socat psmisc ipcalc
    systemctl disable bird bird6 openvpn
    ```


2. Clone the `fabric` branch of the PEERING client repository

   ```bash
   git clone https://github.com/PEERINGTestbed/client.git --branch fabric
   ```

   > The FABRIC branch is identical to the main branch, it just changes the configuration files to allow connection to the FABRIC routers instead of PEERING's normal routers.


3. Configure the PEERING client

   a. Copy your certificates into the `client/certs` directory
   
   b. Ensure certificates do not have group or other read permissions (run `chmod 400` to fix)
   
   c. Add your PEERING-allocated IPv4 and IPv6 prefixes to `client/prefixes.txt` and `client/prefixes6.txt`, respectively, one prefix per line.
   
   > Your certificates and your assigned prefixes can be obtained from the [PEERING Dashboard](https://peering.ee.columbia.edu/account/) page after logging in.


4. Start using your PEERING client

### Node Autoconfiguration

The code cells below will attempt to automatically perform steps 1-3 above.  If something breaks, debugging over SSH may be necessary.  Please update the `PREFIXES4` and `PREFIXES6` variables in the first cell, and upload your certificate into `peering_files/client.crt` and your key to `peering_files/client.key`.

In [9]:
PREFIXES4 = [
    "10.0.0.0/24",
    "10.0.1.0/24",
]
PREFIXES6  = [
    "dead:beef:0000::/48",
    "dead:beef:0001::/48",
]
for filename in ["client.crt", "client.key"]:
    if not os.path.exists(f"peering_files/{filename}"):
        assert False, "Please upload your certificate and key into peering_files"
NAT64_SCRIPT = pathlib.Path("../fablib_api/accessing_ipv4_services_from_ipv6_nodes/nat64.sh").expanduser()
if not os.path.exists(NAT64_SCRIPT):
    assert False, f"NAT64 script not found at {NAT64_SCRIPT}"
os.makedirs("peering_files/generated", exist_ok=True)
with open("peering_files/generated/prefixes.txt", "w", encoding="utf8") as fd:
    fd.write("\n".join(PREFIXES4))
with open("peering_files/generated/prefixes6.txt", "w", encoding="utf8") as fd:
    fd.write("\n".join(PREFIXES6))

In [ ]:
home, _stderr = node.execute("echo -n $HOME")
node = slice.get_node(name=node_name)
node.upload_file(NAT64_SCRIPT, f"{home}/nat64.sh")
node.upload_file('./script/node-autoconfig.sh', f'{home}/node-autoconfig.sh')
node.execute(f"chmod 755 /home/ubuntu/nat64.sh {home}/node-autoconfig.sh")
node.execute(f"{home}/nat64.sh")
node.execute("git clone --branch fabric https://github.com/PEERINGTestbed/client.git")
node.upload_file('peering_files/client.crt', f'{home}/client/certs/client.crt')
node.upload_file('peering_files/client.key', f'{home}/client/certs/client.key')
node.upload_file('peering_files/generated/prefixes.txt', f'{home}/client/prefixes.txt')
node.upload_file('peering_files/generated/prefixes6.txt', f'{home}/client/prefixes6.txt')
node.execute(f"{home}/node-autoconfig.sh")

## Start using your PEERING client

Once your client is configured you should be able to run experiments.  Here are some steps to get you started:

- Access the VM using SSH (see [FABRIC's documentation](../../configure.ipynb))

- `cd` into the client directory

- Use the `./peering` script to establish an OpenVPN connection to a PEERING router, e.g., `./peering openvpn up fabwash`

- Use the `./peering` script to establish a BGP session for exchanging routes with the router, e.g., `./peering bgp start`.

    > Warning: When using IPv6, you need to edit the `client/bird6/bird6.conf` file and set a valid router ID (search for the line starting with `router id`) and use a unique IP address (e.g., one allocated to your experiment)

Read the [PEERING client's README file](https://github.com/PEERINGTestbed/client/blob/master/README.md) and the output of `./peering prefix` to find out how to make and control announcements.  For example, to announce your prefix out of all PEERING routers you are connected to, use `./peering prefix announce <prefix>`.

You can check that your prefix is propagating by using Looking Glass servers from multiple providers:

- [Spring](https://www.sprint.net/tools/looking-glass)
- [NTT](https://www.gin.ntt.net/looking-glass-landing/)
- [Level3/Lumen](https://lookingglass.centurylink.com/)

> Troubleshooting: If the  prefix does not seem to be propagating, check that the OpenVPN tunnel is up, that the BGP session is established, and that your client is exporting the prefix to the desired router:
>
> ```bash
> ./peering openvpn status
> ./peering bgp status
> ./peering bgp adv <router>
> ```

To shut everything up, stop the BGP server and OpenVPN tunnels:

```bash
./peering bgp stop
./peering openvpn down all
```

## PEERING Deployment on Fabric

These are the currently-deploymed PEERING routers on FABRIC.  The provided hostnames resolve to the IPv6 FABnet6 addresses your client's OpenVPN uses to connect to the router from within FABRIC.  The FABnet6 addresses are not reachable from outside FABRIC by default.  These hostnames do not have IPv4 addresses; use `dig AAAA <hostname>` to resolve them.

* Washington, DC: `fabwash.atlas.peering.ee.columbia.edu`
* StarLight, IN: `fabstar.atlas.peering.ee.columbia.edu`